# <span style = "color:red;">Estimating the income bracket</span>

In [1]:
from sklearn import preprocessing 
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [2]:

input_file = 'adult.data.txt'

# Reading the data

X = []
y = []

count_lessthan50k = 0
count_morethan50k = 0
num_images_threshold = 30000

with open(input_file, 'r') as f:
    for line in f.readlines():
        
        if '?' in line:
            continue

        data = line[:-1].split(', ')

        if data[-1] == '<=50K' and count_lessthan50k < num_images_threshold:
            X.append(data)
            count_lessthan50k = count_lessthan50k + 1

        elif data[-1] == '>50K' and count_morethan50k < num_images_threshold:
            X.append(data)
            count_morethan50k = count_morethan50k + 1

        if count_lessthan50k >= num_images_threshold and count_morethan50k >= num_images_threshold:
            break

X = np.array(X)

In [3]:
# Convert string data to numerical data

label_encoder = [] 
X_encoded = np.empty(X.shape)

for i,item in enumerate(X[0]):

    if item.isdigit(): 
        X_encoded[:, i] = X[:, i]
    
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])

X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)


In [4]:
classifier_gaussiannb = GaussianNB()
classifier_gaussiannb.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [5]:
from sklearn.model_selection import cross_val_score, train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.25, random_state = 5)
classifier_gaussiannb = GaussianNB()

In [6]:
classifier_gaussiannb.fit(X_train, y_train)
y_test_pred = classifier_gaussiannb.predict(X_test)

In [7]:
f1 = cross_val_score(classifier_gaussiannb, X, y, scoring = "f1_weighted", cv = 5)

In [8]:
print ("F1 score: " + str(round(100*f1.mean(), 2)) + "%")

F1 score: 75.9%


In [9]:
#Testing encoding on single data instance

input_data = ['39', 'State-gov', '77516', 'Bachelors', '13', 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', '2174', '0', '40', 'United-States'] 
count = 0
input_data_encoded = [-1] * len(input_data)

for i,item in enumerate(input_data):
    
    if item.isdigit():
        input_data_encoded[i] = int(input_data[i])
        
    else:
        input_data_encoded[i] = int(label_encoder[count].transform([input_data[i]]))
        count = count + 1 

input_data_encoded = np.array(input_data_encoded)
input_data_encoded = np.reshape(input_data_encoded, (1, -1))

In [10]:
# Predict and print output for a particular datapoint

output_class = classifier_gaussiannb.predict(input_data_encoded)
print(label_encoder[-1].inverse_transform(output_class)[0])

<=50K
